# ------------------------------------------------------------------------
## Exercise 1:

Within your `codeup-data-science` directory, create a new repo named `spark-exercises`. This will be where you do your work for this module. Create a repository on GitHub with the same name, and link your local repository to GitHub.

Save this work in your `spark-exercises` repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named `spark101` for this exercise.

Create a spark data frame that contains your favorite programming languages.

- Create a dataframe with one column named `language`
> Hint: Start with a pandas dataframe. Maybe use a dictionary?
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

# ------------------------------------------------------------------------

In [6]:
import pyspark

#create spark objeckt
spark = pyspark.sql.SparkSession.builder.getOrCreate()

import pandas as pd
import numpy as np


In [16]:
# create a pandas datadframe before turning it into spark
df_columns = {'language':['python', 'c', 'java', 'rockstar', 'lolcode']}

pandas_dataframe = pd.DataFrame(df_columns)

pandas_dataframe

,language
0,python
1,c
2,java
3,rockstar
4,lolcode


In [17]:
# convert it to a spark dataframe:
df = spark.createDataFrame(pandas_dataframe)
df

DataFrame[language: string]

In [20]:
#view schema of df
df.printSchema()

root
 |-- language: string (nullable = true)



In [18]:
# Spark data frame shape
print("DataFrame shape: ", df.count(), " x ", len(df.columns))

DataFrame shape:  5  x  1


In [19]:
# show first five records 
df.show(5)

+--------+
|language|
+--------+
|  python|
|       c|
|    java|
|rockstar|
| lolcode|
+--------+



# ------------------------------------------------------------------------
## Exercise 2:

Load the `mpg` dataset as a spark dataframe.

a. Create 1 column of output that contains a message like the one below for each record:

    The 1999 audi a4 has a 4 cylinder engine.

> Hint: You will need to concatenate values that already exist in the data with string literals

b. Transform the trans column so that it only contains either manual or auto.

> Hint: Consider spark string methods and `when().otherwise()` chaining
# ------------------------------------------------------------------------

In [24]:
# impor pydataset to upload the mpg dataframe 
from pydataset import data

# store data under the mpg object 
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [25]:
# In order to start using spark SQL, we'll first "register" the table with spark:
mpg.createOrReplaceTempView("mpg")

In [26]:
spark.sql(
    """
SELECT hwy, cty, (hwy + cty) / 2 AS avg
FROM mpg
"""
)

DataFrame[hwy: bigint, cty: bigint, avg: double]

a. Create 1 column of output that contains a message like the one below for each record:

    The 1999 audi a4 has a 4 cylinder engine.

In [30]:
# Note: The pyspark avg and mean functions are aliases of eachother
# import concat function and others 
from pyspark.sql.functions import concat, lit, sum, avg, min, max, count, mean

In [51]:
mpg.select(concat(lit("The "), mpg.year,lit(' '), mpg.manufacturer, lit(' '),  mpg.model, lit(' has a '), mpg.cyl, lit(' cylinder engine.')).alias('Tell me about this car')).show(truncate=False)

+--------------------------------------------------------------+
|Tell me about this car                                        |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

b. Transform the trans column so that it only contains either manual or auto.

> Hint: Consider spark string methods and `when().otherwise()` chaining

In [40]:
from pyspark.sql.functions import when

In [60]:
mpg.select(when(mpg.trans.like("manual%"), "manual").otherwise("automatic")).alias("when + like").show(truncate=False)

+-----------------------------------------------------------+
|CASE WHEN trans LIKE manual% THEN manual ELSE automatic END|
+-----------------------------------------------------------+
|automatic                                                  |
|manual                                                     |
|manual                                                     |
|automatic                                                  |
|automatic                                                  |
|manual                                                     |
|automatic                                                  |
|manual                                                     |
|automatic                                                  |
|manual                                                     |
|automatic                                                  |
|automatic                                                  |
|manual                                                     |
|automat

# ------------------------------------------------------------------------
## Exercise 3: 

Load the `tips` dataset as a spark dataframe.

a. What percentage of observations are smokers?
> Hint: `.groupBy()` and `.withColumn()` are useful functions here

b. Create a column that contains the tip percentage
> Hint: `.withColumn()` is useful here

c. Calculate the average tip percentage for each combination of sex and smoker.
> Hint: Chain additional functions off the answer to part b 

# ------------------------------------------------------------------------

In [63]:
# Load the `tips` dataset as a spark dataframe.
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



a. What percentage of observations are smokers?
> Hint: `.groupBy()` and `.withColumn()` are useful functions here

In [71]:
tips.groupBy("smoker").count().show()

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



b. Create a column that contains the tip percentage
> Hint: `.withColumn()` is useful here

c. Calculate the average tip percentage for each combination of sex and smoker.
> Hint: Chain additional functions off the answer to part b 